# PySceneDetect


In [1]:
import os
import re
import numpy as np
import logging
from scenedetect import  open_video, save_images

# Classes and Types
from scenedetect import StatsManager, AdaptiveDetector
from scenedetect.video_stream import VideoStream
from scenedetect.scene_manager import SceneManager, Interpolation
from scenedetect.scene_detector import SceneDetector
from scenedetect.frame_timecode import FrameTimecode
from typing import List, Tuple, Pattern

In [2]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class DirectoryUtil:
    """Utility class for handling directories."""

    @staticmethod
    def directory_exists(path: str) -> bool:
        """
        Checks if the specified directory exists.
        
        Args:
            path (str): The directory path.
        
        Returns:
            bool: True if the directory exists, False otherwise.
        """
        return os.path.isdir(s=path)

    @staticmethod
    def create_directory(path: str) -> None:
        """
        Creates a directory at the specified path.
        If the directory already exists, no exception is raised.
        
        Args:
            path (str): The directory path.
        """
        try:
            os.makedirs(name=path, exist_ok=True)
            logging.info(msg=f"Directory created: {path}")
        except Exception as e:
            logging.info(msg=f"Error creating directory {path}: {e}")

    @staticmethod
    def ensure_directory(path: str) -> str:
        """
        Ensures that the directory exists. If it does not, the directory is created.
        
        Args:
            path (str): The directory path.

        Return:
            str: Directory path.
        """
        if not DirectoryUtil.directory_exists(path):
            DirectoryUtil.create_directory(path)
            return path
        else:
            logging.info(msg=f"Directory already exists: {path}")
            return path

    @staticmethod
    def find_downloaded_dataset(datasets_path: str, project_id: str, version_number: int) -> str:
        # logging.info(f"Directories in '{datasets_path}': {os.listdir(datasets_path)}")

        expected_name: str = f"{project_id}-{version_number}"
        pattern: Pattern[str] = re.compile(re.escape(pattern=expected_name), re.IGNORECASE)

        for folder in os.listdir(path=datasets_path):
            folder_path: str = os.path.join(datasets_path, folder)

            # logging.info(f"Comparing: '{folder}' vs '{expected_name}'")

            if os.path.isdir(s=folder_path) and pattern.search(string=folder):
                return folder_path 

        logging.error(msg=f"Dataset not found for: '{project_id}-{version_number}' in '{datasets_path}'.")
        raise FileNotFoundError(f"Dataset not found for: '{project_id}-{version_number}' in '{datasets_path}'.")

In [3]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class PySceneDetect:
    """
    A utility class for scene detection in videos.

    Attributes:
        video (VideoStream): The video stream to process.
        scene_manager (SceneManager): The scene manager instance to detect scenes.
        scene_list (Optional[List[Tuple[FrameTimecode, FrameTimecode]]]): 
            A list of detected scenes as tuples of start and end timecodes.
        scenes_detected (bool): A flag indicating whether scenes has been detected.
        interpolation (Optional[Interpolation]): The interpolation method used for image processing.
        valid_extensions (set): A set of valid image file extensions.
    """
    def __init__(self, video: VideoStream) -> None:
        """
        Initialize a PySceneDetect instance.

        Args:
            scene_manager (SceneManager): The scene manager for detecting scenes.
            video (VideoStream): The video stream to be processed.
        """
        self.video: VideoStream = video
        self.scene_manager: SceneManager = SceneManager(stats_manager=StatsManager())
        self.detector: SceneDetector = AdaptiveDetector(
                                                    adaptive_threshold=15.0,
                                                    min_scene_len=20, 
                                                    window_width=8,
                                                    min_content_val=12, 
                                                    luma_only=True, 
                                                    kernel_size=None)
        self.scene_manager.add_detector(detector=self.detector)
        self.scene_list: List[Tuple[FrameTimecode, FrameTimecode]] = []
        self.scenes_detected: bool = False
        self.interpolation: Interpolation = Interpolation(4)
        self.valid_images_extensions = {"jpg", "jpeg", "png", "webp"}

    def set_detector(self, detector: SceneDetector) -> None:
        """
        Adds a scene detector to the scene manager.

        Args:
            detector (SceneDetector): The scene detector to add.
        """
        self.scene_manager.add_detector(detector=detector)
        logging.info(msg="New detector has been set.")

    def set_interpolation(self, interpolation: str) -> None:
        """
        Sets the interpolation method based on the provided string.

        Args:
            interpolation (str): The name of the interpolation method. Supported values are:
                "nearest", "linear", "cubic", "area", and "lanczos".
        """
        interpolation_methods: dict[str, int] = {
            "nearest": 0,
            "linear": 1,
            "cubic": 2,
            "area": 3,
            "lanczos": 4
        }

        if interpolation not in interpolation_methods:
            logging.warning(msg="Interpolation method not recognized. Defaulting to 'lanczos'.")

        self.interpolation = Interpolation(value=interpolation_methods.get(interpolation, 4))

        logging.info(msg="Interpolation has been set.")
                
    def is_image_extension_valid(self, image_extension: str) -> bool:
        """
        Checks if the given image extension is valid.

        Args:
            image_extension (str): The image extension (e.g., ".png").

        Returns:
            bool: True if the extension is valid, False otherwise.
        """
        return image_extension.lower() in self.valid_images_extensions

    def on_new_scene(self, _: np.ndarray, frame_num: int) -> None:
        """
        Callback function that is called when a new scene is detected.

        Args:
            _ (np.ndarray): The image frame where the scene was detected.
            frame_num (int): The frame number where the scene was detected.
        """
        logging.info(msg="Detected Scene: {} - {}".format(frame_num, frame_num + 1))

    def save_images_in_output_dir(self, image_extension: str, output_dir: str, encoder_param: int = 100) -> None:
        """
        Saves images of the detected scenes to the specified output directory.

        Validates:
            - The scene list is set.
            - The image extension is valid.
            - The output directory is not empty.
            - The encoder parameter is within limits.

        Args:
            image_extension (str): The image extension to use when saving images.
            output_dir (str): The directory where images will be saved.
            encoder_param (int, optional): The encoder parameter (default is 100).

        Returns:
            str: An error message if any validation fails; otherwise, images are saved.
        """
        if self.scene_list is None:
            raise ValueError("Scene List is not set. Run set_scene_list() first.")
        if not self.is_image_extension_valid(image_extension):
            raise ValueError("Image extension is not valid. Only 'jpg', 'jpeg', 'png' and 'webp' are valid.")
        if not output_dir.strip():
            raise ValueError("Ouput directory is not set")
        if encoder_param < 0 or encoder_param > 100:
            raise ValueError("Encoder Param is out of limits. Must be higher than 0 and lower than 100")

        save_images(
            video=self.video, 
            scene_list=self.scene_list, 
            output_dir=output_dir, 
            image_extension=image_extension, 
            encoder_param=encoder_param,
            interpolation=self.interpolation
        )
        logging.info(msg="Images has been saved.")

    def detect_scenes(self) -> None:
        """
        Initiates scene detection using the scene manager.

        Sets the 'scenes_detected' flag to True after detection.
        """
        self.scene_manager.detect_scenes(video=self.video, callback=self.on_new_scene)
        self.scenes_detected = True
        self.set_scene_list()
        logging.info(msg="Scenes has been detected.")

    def set_scene_list(self) -> None:
        """
        Retrieves and sets the list of detected scenes from the scene manager.

        Raises:
            ValueError: If scenes has not been detected yet.
        """
        if not self.scenes_detected:
            raise ValueError("Scenes has not been detected yet. Run detect_scenes() first.")

        self.scene_list = self.scene_manager.get_scene_list()
        logging.info(msg="Scene List has been set.")
    
    def print_scenes(self) -> None:
        """
        Shows in logger formated sceneds.

        Validates:
            - The scene list is set.

        Raises:
            ValueError: If scenes has not been detected yet.
        """

        if self.scene_list is None:
            raise ValueError("Scene List is not set. Run set_scene_list() first.")
        
        for i, (start_time, end_time) in enumerate(self.scene_list, start=1):
            logging.info(msg=f"Scene {i}: {start_time.get_timecode()} - {end_time.get_timecode()}")

In [6]:
# Video Path
VIDEO_PATH = '/teamspace/studios/this_studio/test_01.mp4'
# Output Dir Path
OUTPUT_DIR_PATH = '/teamspace/studios/this_studio/detects_scenes'

In [7]:
video: VideoStream = open_video(path=VIDEO_PATH)

In [8]:
pySceneDetect: PySceneDetect = PySceneDetect(video=video)

In [9]:
pySceneDetect.detect_scenes()

2025-03-20 19:45:57,704 - INFO - Detecting scenes...
2025-03-20 19:45:57,925 - INFO - Detected Scene: 37 - 38
2025-03-20 19:45:58,080 - INFO - Detected Scene: 81 - 82
2025-03-20 19:46:06,183 - INFO - Detected Scene: 2347 - 2348
2025-03-20 19:46:12,616 - INFO - Detected Scene: 4286 - 4287
2025-03-20 19:46:38,188 - INFO - Detected Scene: 12111 - 12112
2025-03-20 19:46:38,334 - INFO - Detected Scene: 12160 - 12161
2025-03-20 19:46:42,480 - INFO - Detected Scene: 13347 - 13348
2025-03-20 19:46:45,774 - INFO - Detected Scene: 14376 - 14377
2025-03-20 19:46:48,151 - INFO - Detected Scene: 15185 - 15186
2025-03-20 19:46:48,249 - INFO - Detected Scene: 15218 - 15219
2025-03-20 19:46:48,734 - INFO - Detected Scene: 15374 - 15375
2025-03-20 19:47:00,205 - INFO - Detected Scene: 19145 - 19146
2025-03-20 19:47:05,208 - INFO - Detected Scene: 20879 - 20880
2025-03-20 19:47:05,407 - INFO - Detected Scene: 20944 - 20945
2025-03-20 19:47:16,221 - INFO - Detected Scene: 24635 - 24636
2025-03-20 19:47:1

In [10]:
image_extension: str = 'png'
output_dir: str = DirectoryUtil.ensure_directory(path=OUTPUT_DIR_PATH)

pySceneDetect.save_images_in_output_dir(image_extension=image_extension, output_dir=output_dir)

2025-03-20 19:47:23,245 - INFO - Directory created: /teamspace/studios/this_studio/detects_scenes
2025-03-20 19:47:23,260 - INFO - Saving 3 images per scene [format=png] /teamspace/studios/this_studio/detects_scenes 
2025-03-20 19:48:00,333 - ERROR - Could not generate all output images.
2025-03-20 19:48:00,334 - INFO - Images has been saved.
